# Intan analysis
This notebook will explore the basic pipeline for processing and analyzing neural data recorded using the Intan system.

## Initial setup

In [13]:
import numpy as np
import matplotlib.pyplot as plt
import os, re, glob

import struct

## Preprocess data

In [2]:
raw_dir = 'Y:/James/data/Ephys/02-04-19/intan/j6z4_d25_190204_162928/'

In [10]:
# Placeholders
raw_files = []
ch_ids = []

# Find raw files and associated channel IDs
regexp = re.compile(r'amp-A-[0-9]+')
for f in os.listdir(raw_dir):
    if regexp.search(f):
        raw_files.append(f)
        ch_ids.append(int(f[6:9]))

# Sort if necessary
ch_ids = np.asarray(ch_ids)
sort_idx = np.argsort(ch_ids)
ch_ids = ch_ids[sort_idx]
raw_files = [raw_files[i] for i in sort_idx]

In [3]:
raw_files = sorted(glob.glob(raw_dir + 'amp*.dat'))
ch_ids = np.array([int(f[6:9]) for f in raw_files])

In [4]:
raw_files

['Y:/James/data/Ephys/02-04-19/intan/j6z4_d25_190204_162928\\amp-A-000.dat',
 'Y:/James/data/Ephys/02-04-19/intan/j6z4_d25_190204_162928\\amp-A-001.dat',
 'Y:/James/data/Ephys/02-04-19/intan/j6z4_d25_190204_162928\\amp-A-002.dat',
 'Y:/James/data/Ephys/02-04-19/intan/j6z4_d25_190204_162928\\amp-A-003.dat',
 'Y:/James/data/Ephys/02-04-19/intan/j6z4_d25_190204_162928\\amp-A-004.dat',
 'Y:/James/data/Ephys/02-04-19/intan/j6z4_d25_190204_162928\\amp-A-005.dat',
 'Y:/James/data/Ephys/02-04-19/intan/j6z4_d25_190204_162928\\amp-A-006.dat',
 'Y:/James/data/Ephys/02-04-19/intan/j6z4_d25_190204_162928\\amp-A-007.dat',
 'Y:/James/data/Ephys/02-04-19/intan/j6z4_d25_190204_162928\\amp-A-008.dat',
 'Y:/James/data/Ephys/02-04-19/intan/j6z4_d25_190204_162928\\amp-A-009.dat',
 'Y:/James/data/Ephys/02-04-19/intan/j6z4_d25_190204_162928\\amp-A-010.dat',
 'Y:/James/data/Ephys/02-04-19/intan/j6z4_d25_190204_162928\\amp-A-011.dat',
 'Y:/James/data/Ephys/02-04-19/intan/j6z4_d25_190204_162928\\amp-A-012.dat',

In [5]:
# Channels
shank_1 = np.arange(64, dtype=np.int16)
shank_2 = np.arange(64, 128, dtype=np.int16)

raw_files = [raw_files[i] for i in shank_1]
ch_id = ch_id[shank_1]

In [6]:
# Load data in numpy array
X = []
for i in range(len(raw_files)):
    print('Processing file %02d of %02d...' % (i+1, len(raw_files)))
    X.append(np.fromfile(raw_files[i], dtype=np.int16))
    
X = np.array(X)
print(X.shape)
print('total recording time for this session in mins:  ', X.shape[1]/30000/60)

Processing file 1 of 64...
Processing file 2 of 64...
Processing file 3 of 64...
Processing file 4 of 64...
Processing file 5 of 64...
Processing file 6 of 64...
Processing file 7 of 64...
Processing file 8 of 64...
Processing file 9 of 64...
Processing file 10 of 64...
Processing file 11 of 64...
Processing file 12 of 64...
Processing file 13 of 64...
Processing file 14 of 64...
Processing file 15 of 64...
Processing file 16 of 64...
Processing file 17 of 64...
Processing file 18 of 64...
Processing file 19 of 64...
Processing file 20 of 64...
Processing file 21 of 64...
Processing file 22 of 64...
Processing file 23 of 64...
Processing file 24 of 64...
Processing file 25 of 64...
Processing file 26 of 64...
Processing file 27 of 64...
Processing file 28 of 64...
Processing file 29 of 64...
Processing file 30 of 64...
Processing file 31 of 64...
Processing file 32 of 64...
Processing file 33 of 64...
Processing file 34 of 64...
Processing file 35 of 64...
Processing file 36 of 64...
P

In [9]:
cd D:/james/packages/pyms/mlpy

from mdaio import writemda16i,writemda32

# save data in your desire directory
writemda16i(X,'D:/james/ms-examples/02-04-19/raw1.mda')

D:\james\packages\pyms\mlpy


## Spike sorting
Once the initial `.mda` file is converted from the raw `.dat` files, spikes can be extracted and sorted using `MountainSort`. The final product will be another `.mda` file that contains the firing data.

## Spike analysis

In [35]:
# Load firing data
data_dir = 'Y:/James/data/Ephys/02-04-19/intan/'
filename = 'firings1-r-1'

In [28]:
with open(filename + '.mda', 'rb') as f:
    # First 32 bits = signed integer indicating data type
    code, = struct.unpack('i', f.read(4))
    
    # Second 32 bits: number of bytes per entry
    entry_size, = struct.unpack('i', f.read(4))
    
    # Third 32 bits: number of dimensions
    n_dims, = struct.unpack('i', f.read(4))
    
    # Next 32*n_dims bits: size of each dimension
    shape = np.zeros(n_dims, dtype=np.int32)
    for i in range(n_dims):
        shape[i], = struct.unpack('i', f.read(4))
    
    # Get number of entries
    N = np.prod(shape).astype(np.int32)
    X = np.zeros([N])
    
    # Determine data type from code
    if code == -7: # double
        n_bits = 8
        format_str = 'd'
        
    # Iterate through all entries
    for i in range(N):
        X[i], = struct.unpack(format_str, f.read(n_bits))
    
    # Reshape in Fortran order
    X = X.reshape(shape, order='F') # column-first

In [33]:
# Save as numpy array
np.save(data_dir + filename + '.npy', X)

The shape of the spike array is `[3, n_spikes]`. The rows are as follows:
- `channel_id`: the channel number associated with the spike
- `t_spike`: the sample number associated with the spike
- `spike_id`: the unit number associated with the spike

In [36]:
# Load numpy array
spikes = np.load(data_dir + filename + '.npy')

(3, 4000479)

## Temp code for syncing Intan and Labview